In [ ]:
from itertools import product
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr

In [ ]:
SHOW_PLOTS = False  # Flag to control plot display
SAVE_PLOTS = False  # Flag to save generated plots

In [ ]:
PROJECT_PATH = Path(".")

DATA_PATH = PROJECT_PATH / "data"
DATA_RAW_PATH = DATA_PATH / "01_raw"

IMG_PATH = PROJECT_PATH / "imgs"

CHLOROPHYLL_DATA_PATH = (
    DATA_RAW_PATH / "chl_cmems_obs-oc_glo_bgc-plankton_my_l4-gapfree-multi-4km_P1D.nc"
)

# Data Loading and Preprocessing

The dataset is loaded using xarray. Column names "longitude" and "latitude" are renamed to "lon" and "lat" respectively, simplifying variable access.

## Loading and Initial Inspection


In [ ]:
ds = xr.open_dataset(CHLOROPHYLL_DATA_PATH, engine="netcdf4")
ds = ds.rename({"longitude": "lon", "latitude": "lat"})

lat_min, lat_max = float(ds.lat.min()), float(ds.lat.max())
lon_min, lon_max = float(ds.lon.min()), float(ds.lon.max())
boundaries = (lon_min, lat_min, lon_max, lat_max)
centroid = ((lat_min + lat_max) / 2, (lon_min + lon_max) / 2)

POI = LAT_POI, LON_POI = -23.627534, -70.396026

In [ ]:
# Get the quantiles of chlorophyll-a concentration
q_01, q_99 = ds["CHL"].quantile([0.01, 0.99], method="closest_observation").values
print(f"Quantiles: {q_01 = :.2f}, {q_99 = :.2f}")

## Initial Visualisation

A preliminary visualization of the chlorophyll concentration across the dataset's spatial extent is performed.

In [ ]:
from matplotlib.figure import Figure
from matplotlib.axes import Axes
from matplotlib.colors import Colormap


def plot_chlorophyll(
    ds: xr.Dataset,
    time_index: int,
    fig: Figure = None,
    ax: Axes = None,
    title: str | None = None,
    colorbar: bool = True,
    vmin: float = q_01,
    vmax: float = q_99,
    cmap: str | Colormap | None = None,
) -> tuple[Figure, Axes]:
    """
    Plot chlorophyll-a concentration from a dataset.

    :param ds: xarray dataset containing chlorophyll-a data.
    :param time_index: Index of the time dimension to plot. Must be an integer.
    :param fig: Matplotlib figure object. If None, a new figure is created.
    :param ax: Matplotlib axes object. If None, a new axes is created.
    :param title: Title of the plot. If None, a default title is generated.
    :param colorbar: Whether to display a colorbar. Default is True.
    :param vmin: Minimum value for color mapping. Default is the 1st percentile of chlorophyll-a data.
    :param vmax: Maximum value for color mapping. Default is the 99th percentile of chlorophyll-a data.
    :param cmap: Colormap to use for the plot. Default is "jet".
    :return: Tuple of the figure and axes objects.
    """

    if fig is None or ax is None:
        fig, ax = plt.subplots(figsize=(10, 6))

    lat = ds.lat.data
    lon = ds.lon.data
    chl = ds.CHL.data

    pcolormesh_kwargs = dict(
        shading="auto", cmap=cmap or sns.color_palette("viridis", as_cmap=True)
    )

    pcolormesh_kwargs["vmin"] = vmin
    pcolormesh_kwargs["vmax"] = vmax

    mesh = ax.pcolormesh(lon, lat, chl[time_index, :, :], **pcolormesh_kwargs)

    # Plot the point of interest
    ax.plot(
        LON_POI,
        LAT_POI,
        marker="o",
        color="red",
        markersize=8,
        label="Point of Interest",
    )

    if colorbar:
        colorbar_kwargs = dict(ax=ax, label="Chlorophyll-a concentration (mg/m³)")

        colorbar_kwargs["extend"] = "both"
        cbar = fig.colorbar(mesh, **colorbar_kwargs)
        cbar.cmap.set_under("gray")
        cbar.cmap.set_over("magenta")

    title = (
        title
        or f"Chlorophyll-a concentration on {str(ds.time[time_index].values)[:10]}"
    )

    ax.set_title(title)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")

    ax.set_aspect("equal", adjustable="box")

    return fig, ax

In [ ]:
plot_chlorophyll(ds, 0)
plt.show()

## Research of Missing Values

The initial plot reveals a large, persistent region of missing values (likely land/coast). 

Google Earth Engine is used to visually confirm the geography of this area.

In [ ]:
%%script false --no-raise-error
import ee
import geemap


ee.Authenticate()
ee.Initialize()

region = ee.Geometry.Rectangle(boundaries)


def mask_clouds(image):
    # Select the QA60 band and create a cloud mask
    qa = image.select("QA60")
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    # Apply the cloud mask to the image
    return image.updateMask(mask).divide(10000)


# Select the Sentinel-2 image collection
collection = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(region)
    .filterDate("2021-01-01", "2021-12-31")
    .map(mask_clouds)
)

vis_params = dict(bands=["B4", "B3", "B2"], min=0, max=0.3)

Map = geemap.Map(center=centroid, zoom=6)

Map.addLayer(collection.mosaic().clip(region), vis_params, "Sentinel-2 Copernicus")
Map.addLayer(region, {}, "Region of Interest")
Map.addLayerControl()

Map

As suspected, the systematically missing region corresponds to the coastline. Next, we examine multiple time steps to understand the occurrence of sporadic missing data over the ocean.

In [ ]:
def plot_chlorophyll_grid(
    ds: xr.Dataset,
    n_row: int = 8,
    n_col: int = 6,
    titles: list[str] | None = None,
    figsize: tuple[int, int] = (15, 20),
) -> tuple[Figure, Axes]:
    """
    Plot a grid of the first n_row * n_col chlorophyll-a concentration images
    from a dataset.

    :param ds: xarray dataset containing chlorophyll-a data.
    :param n_row: Number of rows in the grid. Default is 8.
    :param n_col: Number of columns in the grid. Default is 6.
    :return: Tuple of the figure and axes objects.
    """

    if titles is not None:
        assert len(titles) == n_row * n_col, "Number of titles must match grid size."

    fig, axs = plt.subplots(n_row, n_col, figsize=figsize)

    ds = ds.sel(time=ds.time[: n_row * n_col])
    q_01, q_99 = ds["CHL"].quantile([0.01, 0.99], method="closest_observation").values

    for i, j in product(range(n_row), range(n_col)):
        time_index = i * n_col + j

        title = (
            titles[time_index] if titles else f"{str(ds.time[time_index].values)[:10]}"
        )

        plot_chlorophyll(
            ds,
            time_index,
            fig,
            axs[i, j],
            colorbar=False,
            title=title,
            vmin=q_01,
            vmax=q_99,
        )

        if j != 0:
            axs[i, j].set_ylabel("")
            axs[i, j].set_yticklabels([])

        if i != n_row - 1:
            axs[i, j].set_xlabel("")
            axs[i, j].set_xticklabels([])

    cbar = fig.colorbar(
        axs[0, 0].collections[0],
        ax=axs[:, :],
        label="Chlorophyll-a concentration (mg/m³)",
        location="right",
        shrink=0.8,
    )
    cbar.cmap.set_under("gray")
    cbar.cmap.set_over("magenta")

    return fig, axs


plot_chlorophyll_grid(ds, n_row=8, n_col=6)
plt.show()

The grid plot confirms that sporadic missing data exists over the ocean area at various times. To analyze the temporal pattern of missingness, we select a specific coordinate within the ocean and examine its time series.

In [ ]:
chl_da = ds["CHL"].sel(lat=-24, lon=-72, method="nearest")
chl_np = chl_da.values

In [ ]:
# Let's check the number of NaN values in the array
print(
    f"Number of NaN values: {int(np.isnan(chl_np).sum())} / {chl_np.size} "
    f"({float(np.isnan(chl_np).sum() / chl_np.size * 100):.2f}%)"
)

In [ ]:
df = pd.DataFrame({"CHL": chl_np})

msno.matrix(df)
plt.show()

The missingness matrix indicates that most missing values occur at the beginning of the time series. A zoomed-in view will provide more detail.

In [ ]:
msno.matrix(df[:2000])
plt.show()

It can be seen that over the time series there are 24 occasions where there is no data. This may be due to the presence of clouds at the time of image acquisition.

In this case, since there are so few missing values, the values can be interpolated. For the sake of speed and simplicity, linear interpolation will be used. If this is found to affect the result, one can switch to a more robust interpolation, such as spline or linear RBF, albeit sacrificing more computational time for such a small amount of data.

Furthermore, one could use the methods mentioned in the paper [Ferreira, 2021](https://doi.org/10.3390/rs13040675), which mention the 3-step gap-filling algorithm [Sasaoka, 2011](https://doi.org/10.1016/j.rse.2014.05.016), and the 3-week centred moving mean algorithm [Ferreira, 2014](https://doi.org/10.1002/2014JC010323). These methods could be implemented in the future to improve the analysis.

## Missing Data Handling

In [ ]:
# interpolate nan
ds = ds.interpolate_na(dim="time", method="linear")

plot_chlorophyll_grid(ds, n_row=8, n_col=6)
plt.show()

Additionally, the coastal region, identified by persistent missing values (NaNs) even after temporal interpolation, is removed as it is not relevant to the oceanic chlorophyll analysis.

In [ ]:
lon_max = ds.lon.where(~np.isnan(ds.CHL[0, :, :])).max()

ds = ds.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max))

plot_chlorophyll(ds, 0)
plt.show()

In [ ]:
plot_chlorophyll_grid(ds, n_row=8, n_col=6)
plt.show()

Finally, any remaining null values, primarily located in the persistent coastal region identified earlier, must be handled. 

These cannot be interpolated reliably (as they likely represent land or very shallow/complex waters not captured well by the dataset resolution) and filling them with arbitrary values could lead to erroneous conclusions. Therefore, a spatial mask is created to identify valid ocean pixels for analysis.

In [ ]:
mask = ds.CHL.notnull().all(dim="time")
fig, ax = plt.subplots()

mask.plot()
ax.set_aspect("equal", adjustable="box")

# Exploratory Data Analysis

> How is chlorophyll distributed over the course of a year?

To answer this question, the average chlorophyll per month will be calculated and the results plotted.

In [ ]:
def plot_chlorophyll_yearly(
    ds: xr.Dataset, year: str
) -> tuple[Figure, np.ndarray[Axes]]:
    """
    Plot a 12 months average chlorophyll-a concentration from a dataset.

    :param ds: xarray dataset containing chlorophyll-a data.
    :param year: Year to plot. Must be a string in the format "YYYY".
    :return: Tuple of the figure and axes objects.
    """

    ds_ = ds.sel(time=slice(f"{year}-01-01", f"{year}-12-31"))
    ds_ = ds_.resample(time="ME").mean()

    months_names = [
        "January",
        "February",
        "March",
        "April",
        "May",
        "June",
        "July",
        "August",
        "September",
        "October",
        "November",
        "December",
    ]

    rearrange = [0, 3, 6, 9, 1, 4, 7, 10, 2, 5, 8, 11]
    months_names = [months_names[i] for i in rearrange]
    ds_ = ds_.isel(time=rearrange)

    fig, axs = plot_chlorophyll_grid(
        ds_, n_row=3, n_col=4, titles=months_names, figsize=(10, 10)
    )
    fig.suptitle(f"Chlorophyll-a concentration in {year}", fontsize=16)

    return fig, axs


plot_chlorophyll_yearly(ds, "1998")
plt.show()

In [ ]:
def plot_chlorophyll_seasonly(
    ds: xr.Dataset, year: str
) -> tuple[Figure, np.ndarray[Axes]]:
    """
    Plot the 4 seasons in a year.

    :param ds: xarray dataset containing chlorophyll-a data.
    :param year: Year to plot. Must be a string in the format "YYYY".
    :return: Tuple of the figure and axes objects.
    """

    ds_ = ds.sel(time=slice(f"{int(year)-1}-12-21", f"{year}-12-21"))
    ds_ = ds_.resample(time="3MS").mean()

    season_names = ["Summer", "Autumn", "Winter", "Spring"]

    fig, axs = plot_chlorophyll_grid(
        ds_, n_row=2, n_col=2, titles=season_names, figsize=(10, 10)
    )
    fig.suptitle(f"Chlorophyll-a concentration in {year}", fontsize=16)

    return fig, axs


plot_chlorophyll_seasonly(ds, "2000")
plt.show()

Each year will be plotted to observe patterns or anomalies over time.

In [ ]:
year_init, year_end = 2023, 2024
year_init, year_end = 1998, 2024

PATH_PLOT_CHL_YEARLY = IMG_PATH / "chl_yearly"
PATH_PLOT_CHL_SEASONLY = IMG_PATH / "chl_seasonly"
PATH_PLOT_CHL_YEARLY.mkdir(parents=True, exist_ok=True)
PATH_PLOT_CHL_SEASONLY.mkdir(parents=True, exist_ok=True)


# If any option (save or show plots) are eneabled, then run this code
if SAVE_PLOTS or SHOW_PLOTS:
    for year in range(year_init, year_end):
        fig, axs = plot_chlorophyll_yearly(ds, str(year))

        if SAVE_PLOTS:
            fig.savefig(
                PATH_PLOT_CHL_YEARLY / f"chl_yearly_{year}.png", bbox_inches="tight"
            )

        if SHOW_PLOTS:
            plt.show()
        else:
            plt.close(fig)

        fig, axs = plot_chlorophyll_seasonly(ds, str(year))

        if SAVE_PLOTS:
            fig.savefig(
                PATH_PLOT_CHL_SEASONLY / f"chl_seasonly_{year}.png", bbox_inches="tight"
            )

        if SHOW_PLOTS:
            plt.show()
        else:
            plt.close(fig)

The analysis highlights that the highest chlorophyll concentrations occur during the warmer seasons in the studied area of Chile, with the absolute peak typically observed in January. During this month, values can reach the 99th percentile, represented by the magenta color in the visualization. Given that chlorophyll levels act as a proxy for phytoplankton abundance, these findings strongly suggest increased phytoplankton activity during Chile's warmer periods.

The data generally confirms this seasonal pattern for the Chilean location: significant chlorophyll concentrations are typically observed between early spring (September) and late autumn (May). Additionally, there is an observable upward trend in maximum chlorophyll concentrations over the years.

# Time Series Analysis

Primero, buscaremos extraer la serie temporal de la región de interés. En esta serie temporal se construirá a partir de la media, y se realizará para la evolución diaria, mensual y anual, promediando los valores de cada mes y año. Además, se seleccionará una región de interés en torno al punto de interés.

In [ ]:
delta = 2
lon_min_sel, lon_max_sel = LON_POI - delta, LON_POI + delta
lat_min_sel, lat_max_sel = LAT_POI - delta, LAT_POI + delta
lon_min_sel = -71.5
ds = ds.sel(lon=slice(lon_min_sel, lon_max_sel), lat=slice(lat_min_sel, lat_max_sel))

In [ ]:
lon_max = ds.lon.where(~np.isnan(ds.CHL[0, :, :])).max()

ds = ds.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max))

plot_chlorophyll(ds, 0)
plt.show()

In [ ]:
# Daily mean chlorophyll-a concentration
mean_chl_daily = ds.CHL.mean(dim=("lat", "lon"))

# Monthly mean chlorophyll-a concentration
mean_chl_monthly = ds.CHL.resample(time="1ME").mean().mean(dim=("lat", "lon"))

# Yearly mean chlorophyll-a concentration
mean_chl_yearly = ds.CHL.resample(time="1YE").mean().mean(dim=("lat", "lon"))

In [ ]:
def ds_to_df(ds: xr.Dataset, name: str, freq: str) -> pd.DataFrame:
    """
    Convert an xarray dataset to a pandas DataFrame.

    :param ds: xarray dataset to convert.
    :param name: Name of the DataFrame's column.
    :param freq: Frequency to resample the data. Must be a valid pandas frequency string.
    :return: Pandas DataFrame with the data from the xarray dataset.
    """
    df: pd.DataFrame = ds.to_dataframe(name=name)

    df.reset_index(inplace=True)
    df.set_index("time", inplace=True)
    # df = df.asfreq(freq)

    return df

In [ ]:
mean_df_daily = ds_to_df(mean_chl_daily, "mean_chl_daily", "D")
mean_df_monthly = ds_to_df(mean_chl_monthly, "mean_chl_monthly", "ME")
mean_df_yearly = ds_to_df(mean_chl_yearly, "mean_chl_yearly", "YE")

In [ ]:
import statsmodels.tsa.seasonal as sm_seasonal


def plot_seasonal_decompose(
    ts: pd.Series,
    period: int,
    model: str = "additive",
    range_: tuple[int, int] | None = None,
    title: str | None = None,
) -> tuple[sm_seasonal.DecomposeResult, Figure]:
    """
    Decompose a time series into trend, seasonal, and residual components.

    :param ts: Time series to decompose.
    :param period: Period of the seasonal component. Default is 12 for monthly data.
    :param model: Type of decomposition model. Can be 'additive' or 'multiplicative'.
    :param range_: Range of the time series to plot. Default is None (full range).
    :return: Tuple of the decomposition result and the figure object.
    """
    if range_ is not None:
        ts = ts[range_[0] : range_[1]]

    result = sm_seasonal.seasonal_decompose(ts, model=model, period=period)

    fig = result.plot()

    if title is not None:
        # Remove the title from the first axes
        axes = fig.get_axes()
        axes[0].set_title("")

        # Set the title for the entire figure
        fig.suptitle(title, fontsize=12)

    return result, fig

In [ ]:
SAVE_PLOTS = True
SHOW_PLOTS = True

In [ ]:
mean_df_daily["mean_chl_daily"].values

In [ ]:
SEASONAL_DECOMPOSE_PATH = IMG_PATH / "seasonal_decompose"
SEASONAL_DECOMPOSE_PATH.mkdir(parents=True, exist_ok=True)


# (title name, name, time series, period)
time_series_list = [
    (
        "Daily mean chlorophyll-a concentration",
        "mean_daily",
        mean_df_daily["mean_chl_daily"],
        365,
    ),
    (
        "Monthly mean chlorophyll-a concentration",
        "mean_monthly",
        mean_df_monthly["mean_chl_monthly"],
        12,
    ),
    (
        "Yearly mean chlorophyll-a concentration",
        "mean_yearly",
        mean_df_yearly["mean_chl_yearly"],
        None,
    ),
]


if SAVE_PLOTS or SHOW_PLOTS:
    for title, name, ts, period in time_series_list:
        result, fig = plot_seasonal_decompose(
            ts, period=period, model="additive", title=title
        )

        if SAVE_PLOTS:
            fig.savefig(
                SEASONAL_DECOMPOSE_PATH / f"seasonal_decompose_{name}.png",
                bbox_inches="tight",
            )

        if SHOW_PLOTS:
            plt.show()
        else:
            plt.close(fig)

A partir de los gráficos generados se puede observar que la tendencia fue en aumento desde el año 1998 hasta el año 2003. Esta tendencia se mantuvo relativamente estable hasta el año 2022, donde presenta una disminución abrupta.

La estacionalidad se ha mantenido en el tiempo, donde oscila entre los valores de 0.3 y -0.25, aunque esto se pueda deber a la ventana de tiempo que se ha utilizado para realizar el análisis. Para comprobarlo, se realizará una descomposición de la serie temporal con diferentes ventanas de tiempo.

Por último, se observa que el residuo de la serie temporal presenta una alta variabilidad, superando con creces los valores de la tendencia y de la estacionalidad. Esto puede indicar que existe una alta variabilidad en la serie temporal, lo que puede dificultar la identificación de patrones y tendencias a largo plazo. Sin embargo, es importante tener en cuenta que esta variabilidad también puede ser el resultado de factores externos, como cambios climáticos o eventos naturales, que pueden afectar la concentración de clorofila en el océano.

In [ ]:
def plot_seasonal_decompose_range(
    ts_daily: pd.Series,
    ts_monthly: pd.Series,
    year_range: tuple[int, int],
) -> None:
    """
    Plot the seasonal decomposition for daily and monthly time series.

    :param ts_daily: Daily time series to decompose.
    :param ts_monthly: Monthly time series to decompose.
    :param year_range: Range of years to plot. Must be a tuple of two integers.
    :return: None
    """

    year_init, year_end = year_range

    range_daily = ((year_init - 1998) * 365, (year_end - 1998 + 1) * 365)
    range_monthly = ((year_init - 1998) * 12, (year_end - 1998 + 1) * 12)

    title = f"Daily mean chlorophyll-a concentration, period range: {year_init} - {year_end}"
    plot_seasonal_decompose(
        ts_daily, period=365, model="additive", range_=range_daily, title=title
    )
    plt.show()

    title = f"Monthly mean chlorophyll-a concentration, period range: {year_init} - {year_end}"
    plot_seasonal_decompose(
        ts_monthly, period=12, model="additive", range_=range_monthly, title=title
    )
    plt.show()

In [ ]:
plot_seasonal_decompose_range(
    mean_df_daily["mean_chl_daily"], mean_df_monthly["mean_chl_monthly"], (1998, 2012)
)

In [ ]:
plot_seasonal_decompose_range(
    mean_df_daily["mean_chl_daily"], mean_df_monthly["mean_chl_monthly"], (2012, 2020)
)

In [ ]:
plot_seasonal_decompose_range(
    mean_df_daily["mean_chl_daily"], mean_df_monthly["mean_chl_monthly"], (2016, 2024)
)

Se graficaron las series temporales en los rangos del 2004-2012, 2012-2020 y 2016-2024.

Una diferencia importante es el rango de valores que se mueve la componente estacional, donde se observa que en todos los rangos la estacionalidad se mantiene en el rango de 0.5 a -0.5. Sin embargo, los residuos siguen teniendo valores considerablemente altos, al menos, con respecto a la tendencia y la estacionalidad. 

Se observa que la tendencia tiene valores similares a la descomposición de rango completo.

# Spatial Analysis

En esta sección se analizará la distribución espacial de la clorofila a lo largo del tiempo. Para ello, calcularán las métricas expuestas en el paper de Ferreira et al. (2021) y se graficarán los resultados. Las métricas a calcular son las siguientes:

| **Metric** | **Full Name**        | **Unit**         | **Description**                                                  |
|------------|----------------------|------------------|------------------------------------------------------------------|
| Mean       | Chl *a* mean         | mg m⁻³           | Mean Chl *a* concentration of the seasonal cycle                |
| Max        | Chl *a* maximum      | mg m⁻³           | Maximum Chl *a* concentration of the seasonal cycle             |
| BAmp       | Bloom amplitude      | mg m⁻³           | Difference between Chl *a* maximum and mean                     |
| BPeak      | Bloom peak           | -                | DOY of Chl *a* Maximum                                          |
| BInit      | Bloom initiation     | -                | DOY of initiation of the main bloom in the seasonal cycle       |
| BTerm      | Bloom termination    | -                | DOY of termination of the main bloom in the seasonal cycle      |
| BDur       | Bloom duration       | days             | Duration of the main bloom in the seasonal cycle                |
| BArea      | Bloom area           | mg m⁻³           | Biomass of the main bloom in the seasonal cycle                 |
| YArea      | Yearly area          | mg m⁻³           | Total biomass accumulated during the seasonal cycle             |
| BFreq      | Bloom Frequency      | blooms year⁻¹    | Number of blooms in the seasonal cycle                          |


In [ ]:
# def mean_metric(ds: xr.Dataset):
ds = ds.sel(time=slice("1998-01-01", None))

In [ ]:
chl_mean = ds.resample(time="1YE").mean()
chl_mean["CHL"][0].plot()
plt.gca().set_aspect("equal")

In [ ]:
chl_max = ds.resample(time="1YE").max()
chl_max["CHL"][0].plot()
plt.gca().set_aspect("equal")

In [ ]:
b_amp = chl_max - chl_mean
b_amp["CHL"][0].plot()
plt.gca().set_aspect("equal")

In [ ]:
def compute_bloom_initiation(ds: xr.Dataset, year: int) -> xr.Dataset:
    """
    Compute the bloom inition day of year for a year.

    Specifically, the first day when Chl-a rises above a threshold (5% above annual median) and stays there for at least 15 days.

    :param ds: xarray dataset containing chlorophyll-a data.
    :param year: Year to compute the bloom initiation for. Must be an integer.
    :return: xarray dataset with the bloom initiation day of year.
    """

    chl = ds.CHL.sel(time=slice(f"{year}-01-01", f"{year}-12-31"))

    # Chl-a median.
    chl_median = chl.resample(time="1YE").median()  # Dimensions (time=1, lat, lon)
    threshold = chl_median * 1.05
    threshold = threshold[0]  # Dimensions (lat, lon)

    # Threshold mask. Dimensions (time, lat, lon)
    threshold_expanded = threshold.broadcast_like(chl)

    # Values above threshold. Dimensions (time, lat, lon)
    above_threshold = chl > threshold_expanded

    # Number of days above threshold. Dimensions (time, lat, lon)
    rolling_sum = above_threshold.rolling(time=15, center=False).sum()

    # Bloom mask. Dimensions (time, lat, lon)
    bloom_mask = rolling_sum >= 15

    b_init = bloom_mask.argmax(dim="time")

    return b_init


compute_bloom_initiation(ds, 1998).plot()
plt.gca().set_aspect("equal")

In [ ]:
# Bloom threshold

chl_median = ds.resample(time="1YE").median()
threshold = chl_median * 1.05

In [ ]:
ds["CHL"]

In [ ]:
threshold

In [ ]:
threshold.broadcast_like(ds)

In [ ]:
# Bloom initiation
# The day when the bloom starts. Specifically, the first day when Chl-a rises above a threshold (5% above annual median) and stays there for at least 15 days.

# Ensure the dimensions align for broadcasting
threshold_expanded = threshold.broadcast_like(ds)

# Compute a boolean array where values in ds are above the threshold
above_threshold = ds["CHL"] > threshold_expanded


# rolling_sum = above_threshold.rolling(time=15, center=False).sum()


# # bloom_start = above_threshold.rolling(time=15).sum().where(above_threshold).dropna(dim="time", how="all")
# # bloom_start
above_threshold

In [ ]:
chl_mean

In [ ]:
ds.CHL.mean(dim=("time"))